In [1]:
import pandas as pd

In [2]:
turnstile_df = pd.read_csv('stat_turn_m.csv')

In [3]:
turnstile_df.sort_values('STATION_LINE').head()

,Unnamed: 0,STATION_LINE,DAILY_ENTRIES
39,0,1 AV-L,21980.0
156,1,103 ST-1,10333.0
145,2,103 ST-6,10864.0
350,3,103 ST-BC,3914.0
70,4,103 ST-CORONA-7,17518.0


In [18]:
linker_df = pd.read_csv('lookups.csv')

In [19]:
linker_df.sort_values('TurnstileID').head()

,TurnstileID,LocatorIDs
56,1 AV-L,1st Ave L
1,103 ST-1,103rd St 1
2,103 ST-6,103rd St 4-6-6 Express
3,103 ST-BC,103rd St A-B-C
0,103 ST-CORONA-7,103rd St - Corona Plaza 7


In [6]:
hosp_dist_df = pd.read_csv('station_hospital_distances.csv',index_col=0)
hosp_dist_df.head()

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)",0.367354,NEW YORK EYE AND EAR INFIRMARY
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)",0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)",0.588702,MOUNT SINAI WEST
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)",0.686953,BROOKLYN HOSPITAL CENTER
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)",1.134439,BROOKDALE HOSPITAL MEDICAL CENTER


In [7]:
hosp_dist_df['StatID'] = hosp_dist_df['Station Name']+' '+hosp_dist_df['Lines']
hosp_dist_df.head()

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID,StatID
0,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)",0.367354,NEW YORK EYE AND EAR INFIRMARY,Astor Pl 4-6-6 Express
1,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)",0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL,Canal St 4-6-6 Express
2,50th St,1-2,"(40.76172799961419, -73.98384899986625)",0.588702,MOUNT SINAI WEST,50th St 1-2
3,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)",0.686953,BROOKLYN HOSPITAL CENTER,Bergen St 2-3-4
4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)",1.134439,BROOKDALE HOSPITAL MEDICAL CENTER,Pennsylvania Ave 3-4


The code needs to accomplish the following:
For each line in the turnstile dataframe,
Take the STATION_LINE value,
Look up the matching TurnstileID in the linker dataframe,
Take the corresponding LocatorID in the linker dataframe,
Look up the matching StatID in the hospital distance dataframe,
Return the corresponding Hospital Distance (mi) and place that in the turnstile dataframe.

Strategy: will be implemented as a function applied to the turnstile dataframe.
Function will take a string, the linker and hospital distance dataframes.
Will use the ".loc()" functions to retrieve the data at each step.
Convert TurnstileID to the linker's index and StatID to the hospital distance index.

In [20]:
linker_df.set_index('TurnstileID',inplace=True)
linker_df.head()

,LocatorIDs
TurnstileID,
103 ST-CORONA-7,103rd St - Corona Plaza 7
103 ST-1,103rd St 1
103 ST-6,103rd St 4-6-6 Express
103 ST-BC,103rd St A-B-C
104 ST-A,104th St A-S


In [9]:
hosp_dist_df.set_index('StatID',inplace=True)
hosp_dist_df.head()

,Station Name,Lines,Lat / Long,Hospital Distance (mi),Hospital ID
StatID,,,,,
Astor Pl 4-6-6 Express,Astor Pl,4-6-6 Express,"(40.73005400028978, -73.99106999861966)",0.367354,NEW YORK EYE AND EAR INFIRMARY
Canal St 4-6-6 Express,Canal St,4-6-6 Express,"(40.71880300107709, -74.00019299927328)",0.631470,NEW YORK-PRESBYTERIAN/LOWER MANHATTAN HOSPITAL
50th St 1-2,50th St,1-2,"(40.76172799961419, -73.98384899986625)",0.588702,MOUNT SINAI WEST
Bergen St 2-3-4,Bergen St,2-3-4,"(40.68086213682956, -73.97499915116808)",0.686953,BROOKLYN HOSPITAL CENTER
Pennsylvania Ave 3-4,Pennsylvania Ave,3-4,"(40.66471445143568, -73.89488591154061)",1.134439,BROOKDALE HOSPITAL MEDICAL CENTER


In [12]:
def linkup(stat_string,link_df,hosp_df):
    # must check for name in the linker lookup table
    # if not, pass None for this distance
    if stat_string in link_df['LocatorIDs']:
        locator = link_df.loc[stat_string,'LocatorIDs']
    else:
        return None
    dist = hosp_df.loc[locator,'Hospital Distance (mi)']
    return dist

In [21]:
# name the new column according to Natalie's convention
turnstile_df['MIN_HOSP_DIST'] = \
    turnstile_df.STATION_LINE.apply(linkup,link_df=linker_df,hosp_df=hosp_dist_df)


In [22]:
turnstile_df.sort_values('DAILY_ENTRIES',ascending=False).head(60)

,Unnamed: 0,STATION_LINE,DAILY_ENTRIES,MIN_HOSP_DIST
0,433,TIMES SQ 42 ST-1237ACENQRSW,156678.0,1.002665
1,88,34 ST-PENN STA-123ACE,138335.0,1.241920
2,299,GRD CNTRL-42 ST-4567S,120732.0,0.689167
3,86,34 ST-HERALD SQ-BDFMNQRW,106933.0,0.901751
4,26,14 ST-UNION SQ-456LNQRW,84564.0,0.346741
5,290,FULTON ST-2345ACJZ,74761.0,0.101186
6,114,59 ST COLUMBUS-1ABCD,61742.0,0.306962
7,100,47-50 STS ROCK-BDFM,54966.0,0.834593
8,390,PATH NEW WTC-1,48775.0,0.272890
9,115,59 ST-456NQRW,46661.0,0.608687


In [23]:
turnstile_df.sort_values('MIN_HOSP_DIST',ascending=False)

,Unnamed: 0,STATION_LINE,DAILY_ENTRIES,MIN_HOSP_DIST
457,412,ROCKAWAY PARK B-AS,1067.0,4.485908
461,187,BEACH 105 ST-AS,331.0,4.037159
459,194,BEACH 98 ST-AS,695.0,3.640599
460,210,BROAD CHANNEL-AS,480.0,3.343090
453,193,BEACH 90 ST-AS,1493.0,3.245013
...,...,...,...,...
292,159,9TH STREET-1,5161.0,NaN
344,374,NEWARK HM HE-1,4034.0,NaN
377,408,RIT-ROOSEVELT-R,3262.0,NaN
397,407,RIT-MANHATTAN-R,2952.0,NaN


In [25]:
turnstile_df.to_csv('station_hospital_distances.csv',columns=['STATION_LINE','DAILY_ENTRIES','MIN_HOSP_DIST'])